In [1]:
import sys
sys.path.append('..')
import crystalmaths
import numpy as np
#Interactive mode necessary to select planes in image.
%matplotlib qt

# Create ImageHandler

Provide a filepath as the input to the ImageHandler class. Optional argument is window_type. 
Please read about the implications of applying a window to an image, and alternative options. 
The purpose of applying the window is to decrease the contribution of features near the outside of the image, e.g. the scalebar. 


## Q. Why did cropping the scale bar out not remove the streak? 


A: Discontinuities at border being interpreted as features by FFT.

Resources:
https://dsp.stackexchange.com/questions/25927/fourier-transform-artifacts 
https://www.cambridge.org/core/journals/microscopy-and-microanalysis/article/periodic-artifact-reduction-in-fourier-transforms-of-full-field-atomic-resolution-images/80D0E226F0B4B16627AA0B6B9BD24F24

see demo: https://github.com/jacobkimmel/ps_decomp

Also consider:
https://dsp.stackexchange.com/questions/26830/python-scipy-fft-on-numpy-hanning-window-smears-peaks


In [2]:
image_filepath = r'..\Local_data\SL_Bornite_011.png'
image_object = crystalmaths.imagetools.ImageHandler(image_filepath=image_filepath, window_type = 'hanning')
image_object.show_image()

# Interacting with ImageHandler object called image_object

image_object is an object which contains useful attributes used later to perform a search of a crystal structure database.

1. Input the pixel:nm scale ratio for the image. This can be done by either inputting it as an argument to the .get_scale_ratio method, or by keeping the method call blank. The latter opens your image to allow scalebar selection by clicking on the image.
2. Next, we want to select two planes in the image to index. Use the .get_planes() method. This will open up your TEM image and prompt you to click on the planes you are interested in performing a search on. 

In [3]:
image_object.get_scale_ratio(20)

In [4]:
image_object.get_planes()

Pick two planes using left mouse button. Right click once done. Middle mouse button removes most recent point.
Saved points =  [(164.70472617593532, 71.73571054665103), (239.66048059426174, 111.14543709649274)]
Press keyboard button to save points, mouseclick to restart


### Use image_object to Perform Search

result_list = [(pandas dataframe1, pandas dataframe2), ...]
This is a list of tuples where each item is a result from the American Minearologist Crystal Structure Database
- First dataframe contains analysis results, which shows what the possible zone axis and mineral name are
- Second dataframe contains other metadata obtained during the search

In [5]:
result_list = crystalmaths.compile_results.compile_results(
    image_object=image_object, d_spacing_tolerance=0.1, angle_tolerance=10, mineral='Bornite')

Searching this page: http://rruff.geo.arizona.edu/AMS/result.php?diff=vals(1.856587016949773,1.642535392446922),opt(),type(d-spacing),tolerance(0.1)&mineral=Bornite


In [6]:
for result in result_list:
    print(result)

(  Mineral_Name    D-REF1    D-REF2  D-SPACING1  D-SPACING2   H1   K1   L1  \
0    [Bornite]  1.856587  1.642535      1.9337      1.6672  3.0  2.0  3.0   
1    [Bornite]  1.856587  1.642535      1.9337      1.6485  3.0  2.0  3.0   
2    [Bornite]  1.856587  1.642535      1.9337      1.6482  3.0  2.0  3.0   

    H2   K2   L2      angle  angle-fft  angle match         zone axis  
0  1.0 -1.0  1.0  49.664897  56.703261         True  [1.0, 0.0, -1.0]  
1  3.0  2.0  1.0  64.786074  56.703261         True  [-2.0, 3.0, 0.0]  
2  2.0  0.0  0.0  64.759866  56.703261         True  [0.0, 3.0, -2.0]  ,   Mineral_Name    a    b    c  alpha  gamma   beta
0      Bornite  6.7  6.7  6.7  33.53  33.53  33.53)
(  Mineral_Name    D-REF1    D-REF2  D-SPACING1  D-SPACING2   H1   K1   L1  \
1    [Bornite]  1.856587  1.642535      1.9357      1.6682  4.0  0.0  4.0   
3    [Bornite]  1.856587  1.642535      1.9357      1.6484  4.0  0.0  4.0   
5    [Bornite]  1.856587  1.642535      1.9340      1.6693  0.0  8

In [7]:
result_list[0][0]
for i,result in enumerate(result_list):
    if i==0:
        summary_df = result[0].copy(deep=True)
        summary_df.reset_index()
    else:
        summary_df = summary_df.append(result[0], ignore_index=True)
summary_df.drop(columns='angle match', inplace=True)

In [9]:
summary_df['D1-RESIDUAL'] = summary_df.apply(lambda row: (row['D-REF1']-row['D-SPACING1'])**2, axis =1)
summary_df['D2-RESIDUAL'] = summary_df.apply(lambda row: (row['D-REF2']-row['D-SPACING2'])**2, axis =1)
summary_df['ANGLE-RESIDUAL'] = summary_df.apply(lambda row: (row['angle']-row['angle-fft'])**2, axis =1)
summary_df.sort_values(by='ANGLE-RESIDUAL', inplace=True)
summary_df

,Mineral_Name,D-REF1,D-REF2,D-SPACING1,D-SPACING2,H1,K1,L1,H2,K2,L2,angle,angle-fft,zone axis,D1-RESIDUAL,D2-RESIDUAL,ANGLE-RESIDUAL
3,[Bornite],1.856587,1.642535,1.9357,1.6682,4.0,0.0,4.0,3.0,10.0,3.0,49.733539,56.703261,"[-1.0, 0.0, 1.0]",0.006259,0.000659,48.577027
0,[Bornite],1.856587,1.642535,1.9337,1.6672,3.0,2.0,3.0,1.0,-1.0,1.0,49.664897,56.703261,"[1.0, 0.0, -1.0]",0.005946,0.000608,49.538577
5,[Bornite],1.856587,1.642535,1.9340,1.6693,0.0,8.0,4.0,5.0,6.0,3.0,49.659891,56.703261,"[0.0, 1.0, -2.0]",0.005993,0.000716,49.609068
6,[Bornite],1.856587,1.642535,1.9340,1.6505,0.0,8.0,4.0,6.0,4.0,2.0,64.741387,56.703261,"[0.0, 1.0, -2.0]",0.005993,0.000063,64.611470
2,[Bornite],1.856587,1.642535,1.9337,1.6482,3.0,2.0,3.0,2.0,0.0,0.0,64.759866,56.703261,"[0.0, 3.0, -2.0]",0.005946,0.000032,64.908881
1,[Bornite],1.856587,1.642535,1.9337,1.6485,3.0,2.0,3.0,3.0,2.0,1.0,64.786074,56.703261,"[-2.0, 3.0, 0.0]",0.005946,0.000036,65.331867
4,[Bornite],1.856587,1.642535,1.9357,1.6484,4.0,0.0,4.0,2.0,12.0,2.0,64.798955,56.703261,"[-1.0, 0.0, 1.0]",0.006259,0.000034,65.540259
